In [73]:
import numpy as np
import pandas as pd
from nrclex import NRCLex
import translators as ts
from tqdm import tqdm
from progressbar import progressbar
# import stanza as st
# import simplemma
import csv
import re
import string
import fasttext as ft
import translators as ts
import itertools
import re
import spacy 
import en_core_web_lg
nlp = en_core_web_lg.load()
import nltk
nltk.download('punkt')

ft_model = ft.load_model("lid.176.bin")
dir_ = 'BIG_UKR/'
name_uk = dir_+'country_heads_topic_ukraine.xlsx'
name = dir_+'country_heads_topic_all.xlsx'

[nltk_data] Downloading package punkt to /home/kwaks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from tqdm.notebook import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoderLabse = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")

def normalization(embeddings):
    norms = np.linalg.norm(embeddings, 2, axis=1, keepdims=True)
    return embeddings/norms

def val_ct_dict(obj):
    unique, counts = np.unique(obj, return_counts=True)
    val_ct = dict(zip(unique, counts))
    val_ct = dict(reversed(sorted(val_ct.items(), key=lambda item: item[1])))
    return val_ct

def embedd(sentence):
    if isinstance(sentence, str):
        embedding = tf.constant([sentence])
    else:
        embedding = tf.constant(sentence)
    embedding = encoderLabse(preprocessor(embedding))["default"]
    embedding = normalization(embedding)
    return embedding

def embedd_list(list_obj):
    embedded = np.zeros((len(list_obj), 768))
    with tqdm(total=len(list_obj)) as pbar:
        for i in range(len(list_obj)):
            embedded[i] = embedd(list_obj[i])
            pbar.update(1)
    return embedded

def val_ct_dict(obj):
    unique, counts = np.unique(obj, return_counts=True)
    val_ct = dict(zip(unique, counts))
    val_ct = dict(reversed(sorted(val_ct.items(), key=lambda item: item[1])))
    return val_ct

2022-08-07 19:52:35.329571: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-07 19:52:35.329585: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-07 19:52:35.329594: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Kwakuter): /proc/driver/nvidia/version does not exist
2022-08-07 19:52:35.329703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [244]:
# langs =["de", "nl", "da", "fr", "it", "pl", "es", "pt", "ru", "ro", "fi", "lt", "sv", "el", "hr", "ca", "mk"]
langs =[ "pl", "de"]

In [237]:
en = pd.read_csv('lang_sentiment_dicts/en_sentiments.csv')

In [238]:
en

,lem,val
0,abacus,trust
1,abba,positive
2,abbot,trust
3,abduction,"fear,negative,sadness,surprise"
4,abhor,"anger,disgust,fear,negative"
...,...,...
5296,worry,"anticipation,fear,negative,sadness"
5297,wreck,"anger,disgust,fear,negative,sadness,surprise"
5298,wretch,"anger,disgust,negative,sadness"
5299,wrongful,"anger,disgust,negative,sadness"


In [246]:
def get_tag(word, lang):
    return ts.google(word, from_language='en', to_language=lang).split(' ')[0]


In [247]:
sentiment_dicts = []
for lang in langs:
    try:
        print(lang)
        sentiment_dict = dict(zip([get_tag(i,lang) for i in progressbar(en.lem.values )],
                                  [i for i in en.val.values]))
        with open(f'lang_sentiment_dicts/{lang}_sentiments.csv', 'w') as f:
            for key in sentiment_dict.keys():
                f.write("%s,%s\n"%(key,sentiment_dict[key]))

    except Exception as e:
        print(e)

  0% (0 of 5301) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

pl


100% (5301 of 5301) |####################| Elapsed Time: 2:41:22 Time:  2:41:22
  0% (0 of 5301) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

de


100% (5301 of 5301) |####################| Elapsed Time: 2:48:32 Time:  2:48:32


In [233]:
with open("lang_sentiment_dicts/en_embedded.csv",'a') as csvfile:
    x = en.lem.values
    n = 200
    iterations = int(np.ceil(x.shape[0]/n))
    for i in range(iterations):
        print(i,"/",iterations-1)
        batch = x[i*n:i*n+n]
        np.savetxt(csvfile, 
                 embedd(batch), 
                  delimiter=',', newline='\n', header='', footer='', comments='# ', encoding=None)


0 / 26
1 / 26
2 / 26
3 / 26
4 / 26
5 / 26
6 / 26
7 / 26
8 / 26
9 / 26
10 / 26
11 / 26
12 / 26
13 / 26
14 / 26
15 / 26
16 / 26
17 / 26
18 / 26
19 / 26
20 / 26
21 / 26
22 / 26
23 / 26
24 / 26
25 / 26
26 / 26


In [235]:
e

,lem,val
0,abacus,trust
4,abba,positive
5,abbot,trust
6,abduction,"fear,negative,sadness,surprise"
9,abhor,"anger,disgust,fear,negative"
...,...,...
897,worry,"anticipation,fear,negative,sadness"
898,wreck,"anger,disgust,fear,negative,sadness,surprise"
899,wretch,"anger,disgust,negative,sadness"
900,wrongful,"anger,disgust,negative,sadness"
